<a href="https://colab.research.google.com/github/Ufoan/Problema-optimizacion/blob/main/Tarea_informacion_cuantica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paquetes

In [211]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

# Creación de funciones

**Definición matrices de Pauli e identidad:**

In [4]:
X = np.array([[0,1],[1,0]],dtype=complex)
Y = np.array([[0,-1j],[1j,0]],dtype=complex)
Z = np.array([[1,0],[0,-1]],dtype=complex)
I = np.eye(2,dtype=complex)

**Encontamos los vectores propios de las matrices de Pauli y los respectivos proyectores:**

In [71]:
valores_propios_X, vectores_propios_X = np.linalg.eig(X)
valores_propios_Y, vectores_propios_Y = np.linalg.eig(Y)
valores_propios_Z, vectores_propios_Z = np.linalg.eig(Z)
id_mas_X = np.argmin(np.abs(valores_propios_X-1))
id_menos_X = np.argmin(np.abs(valores_propios_X+1))
id_mas_Y = np.argmin(np.abs(valores_propios_Y-1))
id_menos_Y = np.argmin(np.abs(valores_propios_Y+1))
id_mas_Z = np.argmin(np.abs(valores_propios_Z-1))
id_menos_Z = np.argmin(np.abs(valores_propios_Z+1))
mas_X = vectores_propios_X[:,id_mas_X]
menos_X = vectores_propios_X[:,id_menos_X]
mas_Y = vectores_propios_Y[:,id_mas_Y]
menos_Y = vectores_propios_Y[:,id_menos_Y]
mas_Z = vectores_propios_Z[:,id_mas_Z]
menos_Z = vectores_propios_Z[:,id_menos_Z]
Pi_X_mas = np.outer(mas_X,mas_X.conj())
Pi_Y_mas = np.outer(mas_Y,mas_Y.conj())
Pi_Z_mas = np.outer(mas_Z,mas_Z.conj())
Pi_X_menos = np.outer(menos_X,menos_X.conj())
Pi_Y_menos = np.outer(menos_Y,menos_Y.conj())
Pi_Z_menos = np.outer(menos_Z,menos_Z.conj())

In [72]:
Pi_Y_mas

array([[ 0.5+0.j , -0. -0.5j],
       [-0. +0.5j,  0.5+0.j ]])

In [73]:
Pi_Y_menos

array([[0.5+0.j , 0. +0.5j],
       [0. -0.5j, 0.5+0.j ]])

**Crear de manera aleatoria un $\rho$, encontrando aleatoriamente un vector $\vec{a}$ con $\|\vec{a}\|\leq 1$**

In [74]:
def estado_aleatorio():
    a_init = np.random.normal(0,1,3) #dirección aleatoria
    a_init_norm = a_init/np.linalg.norm(a_init) #vector en S2 normalizado, generado aleatoriamente, ahora falta dentro de la bola
    r = np.random.rand()**(1/3) #un radio aleatorio
    a = r * a_init_norm #el a
    rho = 0.5*(I+a[0]*X+a[1]*Y+a[2]*Z)
    return rho



In [75]:
rho_1 = estado_aleatorio()
rho_1

array([[ 0.34752502+0.j        , -0.20679095+0.26089829j],
       [-0.20679095-0.26089829j,  0.65247498+0.j        ]])

**Definimos función para calcular las probabilidades $P(\pm (i))$:**

In [78]:
def probabilidades(rho):
    P_X_mas = np.real_if_close(np.trace(rho@Pi_X_mas))
    P_X_menos = np.real_if_close(np.trace(rho@Pi_X_menos))
    P_Y_mas = np.real_if_close(np.trace(rho@Pi_Y_mas))
    P_Y_menos = np.real_if_close(np.trace(rho@Pi_Y_menos))
    P_Z_mas = np.real_if_close(np.trace(rho@Pi_Z_mas))
    P_Z_menos = np.real_if_close(np.trace(rho@Pi_Z_menos))
    return P_X_mas, P_X_menos, P_Y_mas, P_Y_menos, P_Z_mas, P_Z_menos

In [79]:
probabilidades(rho_1)

(array(0.29320905),
 array(0.70679095),
 array(0.23910171),
 array(0.76089829),
 array(0.34752502),
 array(0.65247498))

**Definimos la función que permite calcular las probabilidades estimadas**:

In [80]:
def probabilidad_estimada(rho,N):
    """
    N: número de veces que se ejecuta el experimento
    """
    P_X_mas, P_X_menos, P_Y_mas, P_Y_menos, P_Z_mas, P_Z_menos = probabilidades(rho)
    mediciones_X = np.random.choice([1,-1], size = N, p = [P_X_mas, P_X_menos])
    mediciones_Y = np.random.choice([1,-1], size = N, p = [P_Y_mas, P_Y_menos])
    mediciones_Z = np.random.choice([1,-1], size = N, p = [P_Z_mas, P_Z_menos])
    conteo_x_mas = np.sum(mediciones_X == 1)
    conteo_x_menos = np.sum(mediciones_X == -1)
    conteo_y_mas = np.sum(mediciones_Y == 1)
    conteo_y_menos = np.sum(mediciones_Y == -1)
    conteo_z_mas = np.sum(mediciones_Z == 1)
    conteo_z_menos = np.sum(mediciones_Z == -1)
    P_X_mas_est = conteo_x_mas/N
    P_X_menos_est = conteo_x_menos/N
    P_Y_mas_est = conteo_y_mas/N
    P_Y_menos_est = conteo_y_menos/N
    P_Z_mas_est = conteo_z_mas/N
    P_Z_menos_est = conteo_z_menos/N
    return P_X_mas_est, P_X_menos_est, P_Y_mas_est, P_Y_menos_est, P_Z_mas_est, P_Z_menos_est


**Definimos función para encontrar los valores estimados de las matrices de pauli:**

In [88]:
def estiimados_pauli(rho,N):
    P_X_mas_est, P_X_menos_est, P_Y_mas_est, P_Y_menos_est, P_Z_mas_est, P_Z_menos_est = probabilidad_estimada(rho,N)
    E_X = np.dot(valores_propios_X,np.array([P_X_mas_est,P_X_menos_est]))
    E_Y = np.dot(valores_propios_Y,np.array([P_Y_mas_est,P_X_menos_est]))
    E_Z = np.dot(valores_propios_Z,np.array([P_Z_mas_est,P_Z_menos_est]))
    return E_X, E_Y, E_Z


In [89]:
def estimacion_matriz_densidad(rho,N):
    E_X, E_Y, E_Z = estiimados_pauli(rho,N)
    rho_estimado = 0.5*(I+E_X*X+E_Y*Y+E_Z*Z)
    return rho_estimado

**Creamos la función para verificar si es un estado cuántico**:

In [90]:
def IsAnState(rho):
    valores_propios, vectores_propios = np.linalg.eig(rho)
    if np.all(valores_propios >= 0):
        return True
    else:
        return False


In [91]:
IsAnState(rho_1)

True

In [92]:
np.trace(rho_1)

np.complex128(1+0j)

In [156]:
IsAnState(estimacion_matriz_densidad(estado_aleatorio(),100))

True

**Ahora hacemos el script que hará la simulación dado un tamaño de ensamble dado:**

In [218]:
def nya(N):
  n_si=0
  n_no=0
  for i in range(N):
    rho = estado_aleatorio()
    if IsAnState(estimacion_matriz_densidad(rho,100)):
      n_si = n_si + 1
    else:
      n_no = n_no + 1
    i = i+1
  return n_si, n_no

In [219]:
nya(10)

(10, 0)

In [220]:
nya(100)

(90, 10)

In [221]:
nya(1000)

(906, 94)

In [222]:
nya(10000)

(9154, 846)

In [223]:
nya(100000)

(91827, 8173)